# Ejercicio: Descenso de Gradiente para Regresión Logística

## Objetivos
En este ejercicio, vas a:
- actualizar el descenso de gradiente para regresión logística.
- explorar el descenso de gradiente en un conjunto de datos conocido

In [ ]:
import copy, math
import numpy as np
#%matplotlib widget
import matplotlib.pyplot as plt
from lab_utils_common import  dlc, plot_data, plt_tumor_data, sigmoid, compute_cost_logistic
from plt_quad_logistic import plt_quad_logistic, plt_prob
plt.style.use('./deeplearning.mplstyle')

## Conjunto de datos
Comencemos con el mismo conjunto de datos de dos X características usado en el ejercicio de frontera de decisión.

In [ ]:
X_train = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y_train = np.array([0, 0, 0, 1, 1, 1])

Como antes, usaremos una función auxiliar para graficar estos datos. Los puntos de datos con etiqueta $y=1$ se muestran como cruces rojas, mientras que los puntos con $y=0$ se muestran como círculos azules.

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,4))
plot_data(X_train, y_train, ax)

ax.axis([0, 4, 0, 3.5])
ax.set_ylabel('$x_1$', fontsize=12)
ax.set_xlabel('$x_0$', fontsize=12)
plt.show()

## Descenso de Gradiente Logístico
<img align="right" src="./images/C1_W3_Logistic_gradient_descent.png"     style=" width:400px; padding: 10px; " >

Recuerda que el algoritmo de descenso de gradiente utiliza el cálculo del gradiente:
$$\begin{align*}
&\text{repetir hasta convergencia:} \; \lbrace \\
&  \; \; \;w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{para j := 0..n-1} \\ 
&  \; \; \;  \; \;b = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \\
&\rbrace
\end{align*}$$

Donde cada iteración realiza actualizaciones simultáneas en $w_j$ para todos los $j$, donde
$$\begin{align*}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{2} \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3} 
\end{align*}$$

* m es el número de ejemplos de entrenamiento en el conjunto de datos      
* $f_{\mathbf{w},b}(x^{(i)})$ es la predicción del modelo, mientras que $y^{(i)}$ es el objetivo
* Para un modelo de regresión logística  
    $z = \mathbf{w} \cdot \mathbf{x} + b$  
    $f_{\mathbf{w},b}(x) = g(z)$  
    donde $g(z)$ es la función sigmoidea:  
    $g(z) = \frac{1}{1+e^{-z}}$   
    


### Implementación del Descenso de Gradiente
La implementación del algoritmo de descenso de gradiente tiene dos componentes: 
- El ciclo que implementa la ecuación (1) anterior. Esto es `gradient_descent` abajo y generalmente se proporciona en ejercicios opcionales y de práctica.
- El cálculo del gradiente actual, ecuaciones (2,3) anteriores. Esto es `compute_gradient_logistic` abajo. Se te pedirá que implementes este ejercicio de la semana.

#### Cálculo del Gradiente, Descripción del Código
Implementa las ecuaciones (2),(3) anteriores para todos los $w_j$ y $b$.
Hay muchas formas de implementarlo. A continuación se describe esta:
- inicializar variables para acumular `dj_dw` y `dj_db`
- para cada ejemplo
    - calcular el error para ese ejemplo $g(\mathbf{w} \cdot \mathbf{x}^{(i)} + b) - \mathbf{y}^{(i)}$
    - para cada valor de entrada $x_{j}^{(i)}$ en este ejemplo,  
        - multiplicar el error por la entrada  $x_{j}^{(i)}$, y sumar al elemento correspondiente de `dj_dw`. (ecuación 2 anterior)
    - sumar el error a `dj_db` (ecuación 3 anterior)

- dividir `dj_db` y `dj_dw` por el número total de ejemplos (m)
- nota que $\mathbf{x}^{(i)}$ en numpy es `X[i,:]` o `X[i]`  y $x_{j}^{(i)}$ es `X[i,j]`

In [ ]:
def compute_gradient_logistic(X, y, w, b): 
    """
    Calcula el gradiente para regresión logística 
 
    Args:
      X (ndarray (m,n): Datos, m ejemplos con n características
      y (ndarray (m,)): valores obejtivo
      w (ndarray (n,)): parametros del modelo
      b (scalar)      : parametro del modelo
    Returns
      dj_dw (ndarray (n,)): El gradiente del costo w.r.t. los parametros de w. 
      dj_db (scalar)      : El gradiente del costo w.r.t. el parametro b. 
    """
    m,n = X.shape
    dj_dw = np.zeros((n,))                           #(n,)
    dj_db = 0.

    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i],w) + b)          #(n,)(n,)=scalar
        err_i  = f_wb_i  - y[i]                       #scalar
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i,j]      #scalar
        dj_db = dj_db + err_i
    dj_dw = dj_dw/m                                   #(n,)
    dj_db = dj_db/m                                   #scalar
        
    return dj_db, dj_dw  

Verifica la implementación de la función de gradiente usando la siguiente celda.

In [ ]:
X_tmp = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y_tmp = np.array([0, 0, 0, 1, 1, 1])
w_tmp = np.array([2.,3.])
b_tmp = 1.
dj_db_tmp, dj_dw_tmp = compute_gradient_logistic(X_tmp, y_tmp, w_tmp, b_tmp)
print(f"dj_db: {dj_db_tmp}" )
print(f"dj_dw: {dj_dw_tmp.tolist()}" )

**Salida esperada**
``` 
dj_db: 0.49861806546328574
dj_dw: [0.498333393278696, 0.49883942983996693]
```

#### Código de Descenso de Gradiente 
El código que implementa la ecuación (1) anterior se muestra a continuación. Tómate un momento para ubicar y comparar las funciones en la rutina con las ecuaciones anteriores.

In [ ]:
def gradient_descent(X, y, w_in, b_in, alpha, num_iters): 
    """
    Performs batch gradient descent
    
    Args:
      X (ndarray (m,n)   : Datos, m ejemplos con n características
      y (ndarray (m,))   : valores objetivo
      w (ndarray (n,))   : Valores iniciales de los parametros del modelo
      b (scalar)         : Valores iniciales de los parametro del modelo
      alpha (float)      : Tasa de aprendizaje
      num_iters (scalar) : número de iteraciones para correr el descenso gradiente
      
    Returns:
      w (ndarray (n,))   : Valores de parametros actualizados Updated values of parameters
      b (scalar)         : Valor de parametro actualizado 
    """
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):
        # Calculate the gradient and update the parameters
        dj_db, dj_dw = compute_gradient_logistic(X, y, w, b)   

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               
        b = b - alpha * dj_db               
      
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( compute_cost_logistic(X, y, w, b) )

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteracion {i:4d}: Costo {J_history[-1]}   ")
        
    return w, b, J_history         #return final w,b and J history for graphing


Ejecutemos el descenso de gradiente en nuestro conjunto de datos.

In [ ]:
w_tmp  = np.zeros_like(X_train[0])
b_tmp  = 0.
alph = 0.1
iters = 10000

w_out, b_out, _ = gradient_descent(X_train, y_train, w_tmp, b_tmp, alph, iters) 
print(f"\nparametros actualizados: w:{w_out}, b:{b_out}")

#### Grafiquemos los resultados del descenso de gradiente:

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,4))
# plot the probability 
plt_prob(ax, w_out, b_out)

# Plot the original data
ax.set_ylabel(r'$x_1$')
ax.set_xlabel(r'$x_0$')   
ax.axis([0, 4, 0, 3.5])
plot_data(X_train,y_train,ax)

# Plot the decision boundary
x0 = -b_out/w_out[0]
x1 = -b_out/w_out[1]
ax.plot([0,x0],[x1,0], c=dlc["dlblue"], lw=1)
plt.show()

En el gráfico anterior:
 - el sombreado refleja la probabilidad y=1 (resultado antes de la frontera de decisión)
 - la frontera de decisión es la línea donde la probabilidad = 0.5
 

## Otro conjunto de datos
Volvamos a un conjunto de datos de una sola variable. Con solo dos parámetros, $w$, $b$, es posible graficar la función de costo usando un gráfico de contorno para tener una mejor idea de lo que hace el descenso de gradiente.

In [ ]:
x_train = np.array([0., 1, 2, 3, 4, 5])
y_train = np.array([0,  0, 0, 1, 1, 1])

Como antes, usaremos una función auxiliar para graficar estos datos. Los puntos de datos con etiqueta $y=1$ se muestran como cruces rojas, mientras que los puntos con $y=0$ se muestran como círculos azules.

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,3))
plt_tumor_data(x_train, y_train, ax)
plt.show()

En el gráfico a continuación, prueba:
- cambiar $w$ y $b$ haciendo clic dentro del gráfico de contorno en la parte superior derecha.
    - los cambios pueden tardar uno o dos segundos
    - observa el valor cambiante del costo en el gráfico superior izquierdo.
    - observa que el costo se acumula por una pérdida en cada ejemplo (líneas punteadas verticales)
- ejecuta el descenso de gradiente haciendo clic en el botón naranja.
    - observa el costo disminuyendo constantemente (el gráfico de contorno y el de costo están en log(costo))
    - hacer clic en el gráfico de contorno reiniciará el modelo para una nueva ejecución
- para reiniciar el gráfico, vuelve a ejecutar la celda

In [ ]:
w_range = np.array([-1, 7])
b_range = np.array([1, -14])
quad = plt_quad_logistic( x_train, y_train, w_range, b_range )

## ¡Felicitaciones!
Has:
- examinado las fórmulas e implementación del cálculo del gradiente para regresión logística
- utilizado esas rutinas en
    - exploración de un conjunto de datos de una sola variable
    - exploración de un conjunto de datos de dos variables